# Global Sensitivity Analysis

This is a more complex case. Involves varying multiple parameters together and understanding which parameter is the one with the largest influence on results. This translates into a large simulation testing the various possibilities. One could use brute force and test all possible combinations of all parameters, and then run a regression on the results where the dependent variable is the impact of the system and the independent variables are the parameters. However, calculating a result for every possible combination might end up taking too much computation time, especially if the number of tested variables is high. Therefore, we use the Saltelli approah and a library called SALib wich is made for this purpose.

In [1]:
# Import brightway2.5 packages
import bw_processing as bwp
import bw2calc as bc
import bw2data as bd
import numpy as np
import scipy as sp

In [2]:
bd.projects.set_current('advlca23') # Still working in the same project
bd.databases

Databases dictionary with 5 object(s):
	biosphere3
	ecoinvent 3.9.1 conseq
	exldb
	sa_db
	testdb

In [3]:
#if 'gsa_db' or 'SAtestdb' in bd.databases: del bd.databases['DATABASE NAME'] # to clean up
# bd.databases
# del bd.databases['SAtestdb']
# del bd.databases['gsa_db']

In [4]:
# First make a list of activities, in this case el. markets.
# We want to investigate which of those has the largest influence on the results.

el_markets = [('ecoinvent 3.9.1 conseq', i['code']) 
              for i in bd.Database("ecoinvent 3.9.1 conseq").search('market for electricity, low voltage', limit = 100)]
el_markets[10:15]

[('ecoinvent 3.9.1 conseq', '33b3d1631b310ce2226ee8ea6166fea9'),
 ('ecoinvent 3.9.1 conseq', 'cacc9cd35a80d31fc0827768d00b97cb'),
 ('ecoinvent 3.9.1 conseq', 'de70ea44bf9b18cca8c0b42b37ee2ca4'),
 ('ecoinvent 3.9.1 conseq', '7653b250a4b2a5b621732dd42e584a5f'),
 ('ecoinvent 3.9.1 conseq', 'ee61e56b1386921ce135e3da83e78ccf')]

In [5]:
# let's look at what we actually got here
for m in el_markets[10:15]:
    
    elmarket = bd.Database("ecoinvent 3.9.1 conseq").get(m[1])
    print(elmarket)
    lca = bc.LCA({elmarket: 1}, ('IPCC 2013', 'climate change', 'global warming potential (GWP100)'))
    lca.lci()
    lca.lcia()
    print(lca.score) # The last one is the activity the highest carbon footprint

'market for electricity, low voltage' (kilowatt hour, RS, None)
0.5609360070522222
'market for electricity, low voltage' (kilowatt hour, IE, None)
0.14714303439740414
'market for electricity, low voltage' (kilowatt hour, YE, None)
0.599941477122655
'market for electricity, low voltage' (kilowatt hour, ID, None)
0.6169739166264264
'market for electricity, low voltage' (kilowatt hour, MY, None)
0.5803179147361534


In [6]:
# these activities are parameterized into a bd database
gsa_db = bd.Database('gsa_db')
gsa_db.write({  
    ('gsa_db', 'el_prod_for_sa'): {
        'name': 'Electricity production',
        'unit': 'kilowatt hour',
        'location': 'GLO',
        'exchanges': [{
            'input': ('ecoinvent 3.9.1 conseq', '65b68980a8fbd9d4b37ad546d64ea9bc'), # 'market for electricity, low voltage' (kilowatt hour, JP, None)
            'amount': 0.2,
            'type': 'technosphere',
            'formula': 'par0'}, # one parameter here 
        { 
            'input': ('ecoinvent 3.9.1 conseq', 'f53c71b5459e1fef31cad69bdba93365'), # 'market for electricity, low voltage' (kilowatt hour, UY, None)
            'amount': 0.2,
            'type': 'technosphere',
            'formula': 'par1'}, # one parameter here 
        { 
            'input': ('ecoinvent 3.9.1 conseq', '6074e6459606ce2d43f399ecb2f95038'), # 'market for electricity, low voltage' (kilowatt hour, PH, None)
            'amount': 0.2,
            'type': 'technosphere',
            'formula': 'par2'}, # one parameter here 
        { 
            'input': ('ecoinvent 3.9.1 conseq', 'c80021ca85117d7ed6a5f348af3ec97f'), # 'market for electricity, low voltage' (kilowatt hour, LT, None)
            'amount': 0.2,
            'type': 'technosphere',
            'formula': 'par3'}, # one parameter here 
        { 
            'input': ('ecoinvent 3.9.1 conseq', '22c7aa161aa6118889763b099d486e09'), # 'market for electricity, low voltage' (kilowatt hour, GE, None)
            'amount': 0.2,
            'type': 'technosphere',
            'formula': 'par4'}, # one parameter here 
        {
            'input': ('gsa_db', 'el_prod_for_sa'),
            'amount': 1.0,
            'type': 'production'}]}})

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 353.09it/s]


Vacuuming database 


We generate a specific sample for our parameters using the Saltelli approach. 
We will then analyze the results using the Sobol indices.
For this we need functions from the python library SALib. See [here](https://pypi.org/project/SALib/) to install SALib with pip and  [here](https://anaconda.org/conda-forge/salib) to install SALib with Conda (remember that you need to install SALib into your BW2 environment! i.e. first activate the environment and then pip/conda install the SALib library)


In [7]:
# If you don't have SALib, uncomment this line to install.
# ! pip install SALib

In [8]:
# import SALib
from SALib.sample import sobol
from SALib.analyze import sobol as sobol_analyze

In [9]:
from bw2data.parameters import * # we also need the bd parameters

In [10]:
# Step 1. define parameters and their ranges in form of a problem
problem = { 'num_vars': 5, # number of variables
            'names': ['par0', 'par1', 'par2', 'par3', 'par4'], # names of variables, same as parameters
            'bounds': [[0, 1], [0, 1], # I am going to use percentiles for the first two  
                       [0.2*0.9, 0.2*1.1], [0.2*0.9, 0.2*1.1], # uniform distribution for the other three
                       [0.2*0.9, 0.2*1.1]] } 

In [11]:
problem

{'num_vars': 5,
 'names': ['par0', 'par1', 'par2', 'par3', 'par4'],
 'bounds': [[0, 1],
  [0, 1],
  [0.18000000000000002, 0.22000000000000003],
  [0.18000000000000002, 0.22000000000000003],
  [0.18000000000000002, 0.22000000000000003]]}

In [12]:
# Step 2. Generate the samples
param_values = sobol.sample(problem, 8) # 10 samples only, a very small input space. Returns 120 values.
print(param_values.shape)
param_values[0:5] # This is an array object (package numpy) you can think of it as a grid of values 

(96, 5)


array([[0.8123128 , 0.23915494, 0.19089204, 0.18364967, 0.19664472],
       [0.5188967 , 0.23915494, 0.19089204, 0.18364967, 0.19664472],
       [0.8123128 , 0.42148183, 0.19089204, 0.18364967, 0.19664472],
       [0.8123128 , 0.23915494, 0.21930804, 0.18364967, 0.19664472],
       [0.8123128 , 0.23915494, 0.19089204, 0.21124556, 0.19664472]])

There is a caveaut here. The Saltelli method generates uniform distributions. But let's assume the first two parameters of mine are **not** uniformly distributed. So I convert from percentile to value in the distribution.

In [13]:
for i in param_values: # for each list of 5 elements...
    i[0] = sp.stats.norm.ppf(i[0], 0.2, 0.01) # normal distribution
    i[1] = sp.stats.lognorm.ppf(i[1], s = 0.1, scale = 0.2) #lognormal, see here:https://stackoverflow.com/questions/8870982/how-do-i-get-a-lognormal-distribution-in-python-with-mu-and-sigma
param_values[0:5]

array([[0.20886451, 0.18631057, 0.19089204, 0.18364967, 0.19664472],
       [0.20047385, 0.18631057, 0.19089204, 0.18364967, 0.19664472],
       [0.20886451, 0.19607691, 0.19089204, 0.18364967, 0.19664472],
       [0.20886451, 0.18631057, 0.21930804, 0.18364967, 0.19664472],
       [0.20886451, 0.18631057, 0.19089204, 0.21124556, 0.19664472]])

If you need to understand the above in more detail

In [14]:
perc25 = sp.stats.norm.ppf(0.25, 0.2, 0.01) # generate the value of the percentile 0.25  
                                            # of a normal distribution with mean 0.02 and sd 0.01
print(perc25) 


perc50 = sp.stats.norm.ppf(0.5, 0.2, 0.01) # generate the value of the percentile 0.5 (= the mean!!!)  
                                           # of a normal distribution with mean 0.02 and sd 0.01
print(perc50)

0.1932551024980392
0.2


In [15]:
# Step 3. Define the details of each parameter
activity_data = [{'name': 'par0', 'amount': 1.0, 'database': 'gsa_db', 'code' : 'par0code'},
                 {'name': 'par1', 'amount': 1.0, 'database': 'gsa_db', 'code' : 'par1code'},
                 {'name': 'par2', 'amount': 1.0, 'database': 'gsa_db', 'code' : 'par2code'},
                 {'name': 'par3', 'amount': 1.0, 'database': 'gsa_db', 'code' : 'par3code'},
                 {'name': 'par4', 'amount': 1.0, 'database': 'gsa_db', 'code' : 'par4code'}]

parameters.new_activity_parameters(activity_data, "gsagroup", overwrite=True)
parameters.add_exchanges_to_group("gsagroup", gsa_db.get('el_prod_for_sa'))
ActivityParameter.recalculate_exchanges("gsagroup")
for param in ActivityParameter.select():
    print(param, param.amount)

Activity parameter: my group:__dummy_88fb3a8e9ed14a31a87aa3ba258116aa__ 0.0
Activity parameter: my group:my_parameter 0.45
Activity parameter: gsagroup:par0 1.0
Activity parameter: gsagroup:par1 1.0
Activity parameter: gsagroup:par2 1.0
Activity parameter: gsagroup:par3 1.0
Activity parameter: gsagroup:par4 1.0
Activity parameter: gsagroup:__dummy_9a188f0e71e94352ad3ea60ba3fba5fa__ 0.0


In [16]:
# Still step 3. Run these samples in our model and store the results in a list. It will take a while
GSA_el_value_results = []
for v in param_values:
    ActivityParameter.update(amount = v[0]).where(ActivityParameter.name == 'par0').execute()
    ActivityParameter.update(amount = v[1]).where(ActivityParameter.name == 'par1').execute()
    ActivityParameter.update(amount = v[2]).where(ActivityParameter.name == 'par2').execute()
    ActivityParameter.update(amount = v[3]).where(ActivityParameter.name == 'par3').execute()
    ActivityParameter.update(amount = v[4]).where(ActivityParameter.name == 'par4').execute()
    
    ActivityParameter.recalculate_exchanges("gsagroup")
    
    lca = bc.LCA({gsa_db.get('el_prod_for_sa') : 1}, ('IPCC 2013', 'climate change', 'global warming potential (GWP100)'))
    lca.lci()
    lca.lcia()
    GSA_el_value_results.append(lca.score)

In [17]:
# Step 4. Feed the problem and the results to the Sobol function to obtain the Sobol indices 
Si = sobol_analyze.analyze(problem, np.array(GSA_el_value_results), print_to_console=True) # must use np.array
# Returns a dictionary with keys 'S1', 'S1_conf', 'ST', and 'ST_conf'
# (first and total-order indices with bootstrap confidence intervals)

            ST   ST_conf
par0  0.000227  0.000304
par1  0.253826  0.285700
par2  0.185187  0.213310
par3  0.000755  0.000774
par4  0.126206  0.190443
            S1   S1_conf
par0 -0.009971  0.014886
par1  0.332816  0.489820
par2  0.308072  0.597460
par3  0.018242  0.034352
par4  0.169177  0.471216
                    S2   S2_conf
(par0, par1)  0.023407  0.042040
(par0, par2)  0.014197  0.028145
(par0, par3)  0.013770  0.035408
(par0, par4)  0.010425  0.033453
(par1, par2) -0.297103  1.183063
(par1, par3) -0.227598  1.091393
(par1, par4) -0.091024  1.252853
(par2, par3) -0.135969  0.739566
(par2, par4) -0.151741  0.594722
(par3, par4) -0.010025  0.048172


/opt/miniconda3/envs/bw2.5/lib/python3.11/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


How to interpret this? S1 is the **first-order sensitivity index** (or main effect index), i.e. is the ration between the variance associated with the parameter and the total model variance. It is the *fraction* of total variance that is explained by the parameter. Or, in better words: _it measures the effect of varying the parameter alone, but averaged over variations in other input parameters_ ([See here](https://en.wikipedia.org/wiki/Variance-based_sensitivity_analysis)). In our case _par1_ alone explains 21% of the total variance. And seems therefore the most important one - or better, the one to which results are most sensitive to - , followed by _par2_ and _par3_.

ST instead is the **total-effect index** (or total-order index), i.e. tells how much of the total variance is explained by the parameter taking into account also all its higher order interactions (e.g. the interaciton between _par1_ * _par3_, etc.) Also this case the most important parameter is _par1_ and then secondary parameters are _par2_and _par3_. We now konw that _par2_ has not only a large effect but also a large effect in interaction with other parameters.

However, before we draw some conclusions, let's do two things: first, check the confidence intervals ofr each index. They are very large! So we can't really have much confience in these results...Second, check the next cell...

In [18]:
sum(Si['S1']) # Different from 1, so this is not a purely additive model...but is it really true? 
# Or is it just because we didn't have enough simulations?

0.8183353811663734

# Understanding the Sobol indices even better

Let's use an even simpler database, increase the number of simulations, and look at the indices again.
Below here I redo the same but with an even more articifial example, a foreground-only system with only two emissions of equal value. Each of them is parametrised. This ligher system should allow for a faster simulation.

Since the emissions have equal amount and equal carachterisation factor, we expect that the results will be equally sensitive to each of those. In oher words, these two parameters have equal importance and the sensitivity analysis results should show this.

In [19]:
t_db = bd.Database('SAtestdb')

t_db.write({
    ("SAtestdb", "Electricity production"):{
        'name':'Electricity production',
        'unit': 'kWh', 
        'location': 'GLO',
        'exchanges': [{
                'input': ('SAtestdb', 'Carbon dioxide'),
                'amount': 1,
                'unit': 'kg',
                'type': 'biosphere',
                'formula': 'parCO2' # one parameter here
            },{
                'input': ('SAtestdb', 'Sulphur dioxide'),
                'amount': 1, # same value
                'unit': 'kg',
                'type': 'biosphere',
                'formula': 'parSO2' # other parameter here
            },{
                'input': ('SAtestdb', 'Electricity production'), #important to write the same process name in output
                'amount': 1,
                'unit': 'kWh',
                'type': 'production'
            }]
        },
    ('SAtestdb', 'Carbon dioxide'):{'name': 'Carbon dioxide', 'unit':'kg', 'type': 'biosphere'},
    ('SAtestdb', 'Sulphur dioxide'):{'name': 'Sulphur dioxide', 'unit':'kg', 'type': 'biosphere'}
   
    })

myLCIAdata = [[('SAtestdb', 'Carbon dioxide'), 1.0], 
              [('SAtestdb', 'Sulphur dioxide'), 1.0]]

imaginaryLCIAmethod = ('just', 'another', 'LCIAmethod')
my_method = bd.Method(imaginaryLCIAmethod)
my_method.validate(myLCIAdata)
my_method.register() 
my_method.write(myLCIAdata)
my_method.load()


lca = bc.LCA({t_db.get('Electricity production') : 1}, imaginaryLCIAmethod)
lca.lci()
lca.lcia()
print('-----\n-----\n-----\n-----\nlca score =', lca.score)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 6506.16it/s]


Vacuuming database 
-----
-----
-----
-----
lca score = 2.0


In [20]:
# Step 1. define parameters and their ranges in form of a problem
problem = { 'num_vars': 2, 
            'names': ['parCO2', 'parSO2'], 
            'bounds': [[0.9, 1.1], [0.9, 1.1]] }  # 10 % variation

# Step 2. Generate the samples
param_values = sobol.sample(problem, 1000)
print(param_values.shape)
param_values[0:5]

# Step 3. Define the details of the parameter 'my_parameter'
activity_data2 = [{'name': 'parCO2', 'amount': 1.0, 'database': 'SAtestdb', 'code' : 'parCO2code'},
                 {'name': 'parSO2', 'amount': 1.0, 'database': 'SAtestdb', 'code' : 'parSO2code'}]

parameters.new_activity_parameters(activity_data2, "my group 2", overwrite=True)
parameters.add_exchanges_to_group("my group 2", t_db.get('Electricity production'))
ActivityParameter.recalculate_exchanges("my group 2")
    
GSA_el_value_results = []
for v in param_values:
    ActivityParameter.update(amount = v[0]).where(ActivityParameter.name == 'parCO2').execute()
    ActivityParameter.update(amount = v[1]).where(ActivityParameter.name == 'parSO2').execute()
    
    ActivityParameter.recalculate_exchanges("my group 2")
    
    lca = bc.LCA({t_db.get('Electricity production') : 1}, imaginaryLCIAmethod)
    lca.lci()
    lca.lcia()
    GSA_el_value_results.append(lca.score)

# Step 4. Feed the problem and the results to the Sobol function to obtain the Sobol indices 
Si = sobol_analyze.analyze(problem, np.array(GSA_el_value_results), print_to_console=True) # must use np.array
# Returns a dictionary with keys 'S1', 'S1_conf', 'ST', and 'ST_conf'
# (first and total-order indices with bootstrap confidence intervals)

(6000, 2)


/opt/miniconda3/envs/bw2.5/lib/python3.11/site-packages/scipy/stats/_qmc.py:958: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)


              ST   ST_conf
parCO2  0.501104  0.039914
parSO2  0.501619  0.044405
              S1   S1_conf
parCO2  0.497769  0.059650
parSO2  0.498317  0.053126
                        S2  S2_conf
[parCO2, parSO2]  0.006671  0.10532


/opt/miniconda3/envs/bw2.5/lib/python3.11/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


Easier to interpret. The first order indices have almost the same value. Their error is much lower. The Total effect indices are very close to the first order one. because there is no interaction - and for the same reason the second order indices are close to zero. So everything works as expected and the two parameters are equally important. Moreover...

In [21]:
sum(Si['S1']) # close to 1, so this confirms it is a purely additive model

0.9960862445801622

For further understanding on how the Saltelli sampling method works, and how to correctly interpret the Sobol indices, refer to Chapter 4: _Variance based methods_  in Saltelli (2008). Otherwise check the [wikipedia page on variance-based sensitivity analysis](https://en.wikipedia.org/wiki/Variance-based_sensitivity_analysis) which contains a reasonably good summary on Sobol indices and the Saltelli sampling method. 

_Saltelli, A., Ratto, M., Andres, T., Campolongo, F., Cariboni, J., Gatelli, D., Saisana, M., Tarantola, S., 2008. Global Sensitivity Analysis. The Primer, Global Sensitivity Analysis. The Primer. [doi:10.1002/9780470725184](https://onlinelibrary.wiley.com/doi/book/10.1002/9780470725184)_


# Trying out a different GSA index

The sobol index is not the only one. We can do GSA with the Borgnonovo index as well, this is supposed to take into account correlation between parameters - and in LCA many parameters are correlatd, e.g. use of coal and production of electricity. You can read more about:
- The index itself in [Borgonovo (2007)](https://doi.org/10.1016/j.ress.2006.04.015): _A new uncertainty importance measure_ 
- [How it is implemented in Salib](https://salib.readthedocs.io/en/latest/api.html#delta-moment-independent-measure) as _Delta Moment-Independent Measure_
- and you can see an applicaiton in GSA for LCA models in [Blanco et al. (2020)](https://doi.org/10.1016/j.jclepro.2020.120968): _Assessing the sustainability of emerging technologies: A probabilistic LCA method applied to advanced photovoltaics_ 

In [22]:
from SALib.sample import latin
from SALib.analyze import delta as delta_analyze

In [23]:
# Step 1. define parameters and their ranges in form of a problem
problem = { 'num_vars': 5, 
            'names': ['par0', 'par1', 'par2', 'par3', 'par4'], 
            'bounds': [[0, 1], [0, 1], # I am going to use percentiles for the first two  
                       [0.2*0.9, 0.2*1.1], [0.2*0.9, 0.2*1.1], # uniform disrribution for the other three
                       [0.2*0.9, 0.2*1.1]] } 

In [24]:
# Step 2. Generate the samples
param_values = latin.sample(problem, 1000) # reduce this number for a fast test simulation!
print(param_values.shape)
param_values[0:5]

(1000, 5)


array([[0.11034264, 0.92784335, 0.21780982, 0.20805709, 0.19520422],
       [0.25325438, 0.03748373, 0.21265009, 0.20894348, 0.18456586],
       [0.60333426, 0.1391949 , 0.21257285, 0.21720184, 0.18654769],
       [0.09159888, 0.16607332, 0.20156005, 0.21540127, 0.20462174],
       [0.08645113, 0.21949643, 0.21045058, 0.19725725, 0.18065   ]])

As before, convert the first two in normal and lognormal respectively

In [25]:
for i in param_values:
    i[0] = sp.stats.norm.ppf(i[0], 0.2, 0.01) #normal
    i[1] = sp.stats.lognorm.ppf(i[1], s = 0.1, scale = 0.2) #lognormal, see here:https://stackoverflow.com/questions/8870982/how-do-i-get-a-lognormal-distribution-in-python-with-mu-and-sigma
param_values[0:5]

array([[0.18775292, 0.23143728, 0.21780982, 0.20805709, 0.19520422],
       [0.19335716, 0.16737738, 0.21265009, 0.20894348, 0.18456586],
       [0.20261987, 0.17945474, 0.21257285, 0.21720184, 0.18654769],
       [0.18669027, 0.18151485, 0.20156005, 0.21540127, 0.20462174],
       [0.18637063, 0.18510586, 0.21045058, 0.19725725, 0.18065   ]])

In [26]:
# Still step 3. Run these samples in our model and store the results in a list
GSA_el_value_results = []
for v in param_values:
    ActivityParameter.update(amount = v[0]).where(ActivityParameter.name == 'par0').execute()
    ActivityParameter.update(amount = v[1]).where(ActivityParameter.name == 'par1').execute()
    ActivityParameter.update(amount = v[2]).where(ActivityParameter.name == 'par2').execute()
    ActivityParameter.update(amount = v[3]).where(ActivityParameter.name == 'par3').execute()
    ActivityParameter.update(amount = v[4]).where(ActivityParameter.name == 'par4').execute()
    
    ActivityParameter.recalculate_exchanges("gsagroup")
    
    lca = bc.LCA({gsa_db.get('el_prod_for_sa') : 1}, ('IPCC 2013', 'climate change', 'global warming potential (GWP100)'))
    lca.lci()
    lca.lcia()
    GSA_el_value_results.append(lca.score)

In [27]:
# Step 4. Feed the problem and the results to the Sobol function to obtain the Sobol indices 
D = delta_analyze.analyze(problem, param_values, np.array(GSA_el_value_results), print_to_console=True) # must use np.array
# Returns a dictionary with keys 'delta', 'delta_conf', 'S1', and 'S1_conf'
# (first and total-order indices with bootstrap confidence intervals)

         delta  delta_conf        S1   S1_conf
par0  0.077885    0.019358  0.006145  0.007342
par1  0.323921    0.021262  0.558370  0.030746
par2  0.119816    0.024221  0.215947  0.041198
par3  0.081587    0.017278  0.013549  0.014616
par4  0.157176    0.021325  0.240379  0.044331


D (Delta) represents the normalized expected shift in the distribution of model output provoked by the parameter under analysis. Delta lies between 0 and 1 and it is zero when the model output is independent of the parameter.

In our case, Delta returns a similar result than the Sobol indices. Again the most sensitive parameter is _par1_ afollowed by _par2_ and _par3_.

### Questions for own reflection and plenum discussion

Could we have found the same results by looking just at the contribution analysis results?
Whn do you think it is usueful to perform this simulation and for which parameters in your LCA model?

## Group exercise (we will do this in class): sensitivity analysis on the case studies.

Reflect about possible source of variance for your case study. Which activities are you unsure about? Are you in doubt about the type of activity used, or about the value used? Which activities do you expect to affect the results? Formulate some hypotheses based on your expectations and your understanding of your product system case. Then select the relevant parameters that could help you test these hypotheses and identify how sensitive are the results of your case study to these. Finally quantify this influence and rank the parameters based on their influence on the results.  